[Wikipedia](https://en.wikipedia.org/wiki/Loop_nest_optimization)

In [1]:
import numpy as np

In [2]:
a_row = 8
a_col = 8
b_row = a_col
b_col = 8
c_row = a_row
c_col = b_col

In [3]:
a = np.random.randint(low=0, high=10, size=(a_row, a_col))
b = np.random.randint(low=0, high=10, size=(b_row, b_col))
c_0 = np.zeros(shape=(c_row, c_col))

In [ ]:
a

In [ ]:
b

In [ ]:
c_0

In [ ]:
c_ref = np.matmul(a, b)
c_ref

In [5]:
def navie(a, b, c):
    for i in range(a_row):
        for j in range(b_col):
            c[i][j] = 0
            for k in range(a_col):
                c[i][j] += a[i][k] * b[k][j]
    return c

In [ ]:
c_navie = navie(a, b, c_0)
c_navie - c_ref

In [6]:
def block_2(a, b, c):
    for i in range(0, a_row, 2):
        for j in range(0, b_col, 2):
            acc00 = acc01 = acc10 = acc11 = 0
            for k in range(a_col):
                acc00 += b[k][j + 0] * a[i + 0][k]
                acc01 += b[k][j + 1] * a[i + 0][k]
                acc10 += b[k][j + 0] * a[i + 1][k]
                acc11 += b[k][j + 1] * a[i + 1][k]
            c[i + 0][j + 0] = acc00
            c[i + 0][j + 1] = acc01
            c[i + 1][j + 0] = acc10
            c[i + 1][j + 1] = acc11
    return c

In [ ]:
c_block_2 = block_2(a, b, c_0)
c_block_2 - c_ref

In [7]:
def block_ib(a, b, c, ib=2):
    for ii in range(0, a_row, ib):
        for j in range(0, b_col, 2):
            for i in range(ii, ii+ib, 2):
                acc00 = acc01 = acc10 = acc11 = 0
                for k in range(a_col):
                    acc00 += b[k][j + 0] * a[i + 0][k]
                    acc01 += b[k][j + 1] * a[i + 0][k]
                    acc10 += b[k][j + 0] * a[i + 1][k]
                    acc11 += b[k][j + 1] * a[i + 1][k]
                c[i + 0][j + 0] = acc00
                c[i + 0][j + 1] = acc01
                c[i + 1][j + 0] = acc10
                c[i + 1][j + 1] = acc11
    return c

In [ ]:
c_block_ib = block_ib(a, b, c_0, ib=4)
c_block_ib - c_ref

In [8]:
def block_ib_kb(a, b, c, ib=2, kb=2):
    for ii in range(0, a_row, ib):
        for kk in range(0, a_col, kb):
            for j in range(0, b_col, 2):
                for i in range(ii, ii+ib, 2):
                    if kk == 0:
                        acc00 = acc01 = acc10 = acc11 = 0
                    else:
                        acc00 = c[i + 0][j + 0]
                        acc01 = c[i + 0][j + 1]
                        acc10 = c[i + 1][j + 0]
                        acc11 = c[i + 1][j + 1]
                    for k in range(kk, kk+kb):
                        acc00 += b[k][j + 0] * a[i + 0][k]
                        acc01 += b[k][j + 1] * a[i + 0][k]
                        acc10 += b[k][j + 0] * a[i + 1][k]
                        acc11 += b[k][j + 1] * a[i + 1][k]
                    c[i + 0][j + 0] = acc00
                    c[i + 0][j + 1] = acc01
                    c[i + 1][j + 0] = acc10
                    c[i + 1][j + 1] = acc11
    return c

In [ ]:
c_block_ib_kb = block_ib_kb(a, b, c_0, ib=4, kb=4)
c_block_ib_kb - c_ref

In [14]:
%timeit np.matmul(a, b)

The slowest run took 10.90 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 1.94 µs per loop


In [9]:
%timeit navie(a, b, c_0)

1000 loops, best of 3: 452 µs per loop


In [10]:
%timeit block_2(a, b, c_0)

1000 loops, best of 3: 308 µs per loop


In [11]:
%timeit block_ib(a, b, c_0, ib=4)

1000 loops, best of 3: 311 µs per loop


In [12]:
%timeit block_ib_kb(a, b, c_0, ib=4, kb=4)

1000 loops, best of 3: 368 µs per loop
